In [ ]:
# Instalação das dependências necessárias
!pip install "tensorflow[and-cuda]" tensorflow-hub
!pip install -q tensorflow tensorflow_hub
!pip install -q scikit-learn pandas numpy tqdm

In [ ]:
# Verificação da instalação do TensorFlow com suporte a GPU
import tensorflow as tf, pprint
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))
pprint.pprint(tf.sysconfig.get_build_info())  # mostra cuda_version e cudnn_version

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
# Carregando os dados
df = pd.read_csv('eventos_artigos_filtrados.csv');
df.shape
#df.head()

In [ ]:
# Preparação dos dados
df = df.drop(columns=['evento_nome', 'resumo'])
df = df.dropna(subset=['titulo_artigo', 'ano_edicao'])
df['antes_2023'] = df['ano_edicao'].apply(lambda x: 1 if x < 2023 else 0)

df["titulo_artigo"] = df["titulo_artigo"].astype(str).str.strip()
df["antes_2023"] = df["antes_2023"].astype(int).values

#df.head()

In [ ]:
#Incluir aqui outros tratamentos, SFC, normalização, etc.

# Teste com Tokenizador ELMo
# Etapas:
- Carregamento do ELMo
- Geração dos embeddings usando ELMo pré-treinado
- Classificador

In [ ]:
# Carregando o modelo ELMo uma única vez usando a API TF2.x
elmo = hub.load("https://tfhub.dev/google/elmo/3")

In [9]:
BATCH_SIZE = 256
todos_embeddings = []

def gerar_embeddings_elmo(textos):
    textos_tensor = tf.constant(textos)
    embeddings_palavras = elmo.signatures['default'](textos_tensor)['elmo']
    
    # Calcula a média dos embeddings das palavras para obter um embedding por frase
    embeddings_frases = tf.reduce_mean(embeddings_palavras, axis=1)
    
    # .numpy() em modo Eager, fora do Keras
    return embeddings_frases.numpy()

In [10]:
titulos = df["titulo_artigo"].tolist()
labels = df["antes_2023"].astype(int).values

for i in tqdm(range(0, len(titulos), BATCH_SIZE)):
    lote_titulos = titulos[i : i + BATCH_SIZE]
    lote_embeddings = gerar_embeddings_elmo(lote_titulos)
    todos_embeddings.append(lote_embeddings)

embeddings_gerados = np.vstack(todos_embeddings)
np.save('embeddings_titulos.npy', embeddings_gerados)
np.save('labels.npy', labels)

print(f"\nEmbeddings gerados para todos os {len(titulos)} títulos.")

100%|██████████| 50/50 [00:16<00:00,  3.05it/s]



Embeddings gerados para todos os 12629 títulos.


# CLASSIFICADOR 

In [11]:
# Carregamento dos Dados Pré-processados ---
X = np.load('embeddings_titulos.npy')
y = np.load('labels.npy')

print(f"Dados carregados. Shape de X: {X.shape}, Shape de y: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Dados divididos: {len(X_train)} para treino, {len(X_test)} para teste.\n")


# Modelo de Classificação ---
model = tf.keras.Sequential([
    # A camada de entrada espera vetores com 1024 dimensões (o tamanho do embedding ELMo)
    tf.keras.layers.InputLayer(input_shape=(1024,)),
    
    # Camada oculta para aprender padrões nos embeddings
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5), # Dropout para evitar overfitting
    
    # Camada de saída para classificação binária
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- Treinamento do Modelo ---
print("\nIniciando o treinamento do classificador...")
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=4,
    validation_data=(X_test, y_test),
    verbose=1
)

# --- Avaliação Final ---
print("\nAvaliação final no conjunto de teste:")
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Acurácia final no teste: {accuracy:.4f}')

Dados carregados. Shape de X: (12629, 1024), Shape de y: (12629,)
Dados divididos: 10103 para treino, 2526 para teste.



/home/ricardo/Mestrado/TAIA/NLP-Embedding/.venv/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,329 (513.00 KB)

...


Iniciando o treinamento do classificador...
Epoch 1/25


2025-09-04 23:03:25.452880: I external/local_xla/xla/service/service.cc:163] XLA service 0x7fbef81b2090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-04 23:03:25.452895: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-09-04 23:03:25.467947: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 188/2526 ━━━━━━━━━━━━━━━━━━━━ 1s 811us/step - accuracy: 0.7118 - loss: 0.6241

I0000 00:00:1757041406.133432  205818 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2526/2526 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7209 - loss: 0.5859 - val_accuracy: 0.7268 - val_loss: 0.5771
Epoch 2/25
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7260 - loss: 0.5689 - val_accuracy: 0.7260 - val_loss: 0.5559
Epoch 3/25
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7312 - loss: 0.5580 - val_accuracy: 0.7284 - val_loss: 0.5507
Epoch 4/25
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7294 - loss: 0.5547 - val_accuracy: 0.7276 - val_loss: 0.5488
Epoch 5/25
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7325 - loss: 0.5468 - val_accuracy: 0.7292 - val_loss: 0.5507
Epoch 6/25
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7341 - loss: 0.5446 - val_accuracy: 0.7304 - val_loss: 0.5972
Epoch 7/25
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7345 - loss: 0.5353 - val_accuracy: 0.7316 - val_loss: 0.5592
Epoch 8/25
2526/2526 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7412 - loss: 0.5291 - val_accurac

In [12]:
model.save('classificador_com_ELMo.keras')